**Disclaimer**: This agent is not intended as financial advice.  It is for informational and entertainment purposes only.  Do your own due diligence.

In [1]:
!pip install -U -q langgraph langchain langchain_openai langchainhub polygon-api-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 kB 977.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.6/271.6 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.6/86.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.5/162.5 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [2]:
import getpass
import os

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [3]:
# You can get an API key here https://polygon.io/
os.environ["POLYGON_API_KEY"] = getpass.getpass()

··········


# Define the tools our agent can use

In [4]:
from langchain.tools import BaseTool, StructuredTool, tool

@tool
def discounted_cash_flow(
    free_cash_flow: float,
    growth_rate: float = 0.05,
    discount_rate: float = 0.10,
    terminal_growth_rate: float = 0.02,
    num_years: int = 5,
  ) -> float:
    """
    Computes the discounted cash flow (DCF) for a given company based on the current free cash flow.
    Use this function to calculate the intrinsic value of a stock.
    """
    # Estimate the future cash flows based on the growth rate
    cash_flows = [free_cash_flow * (1 + growth_rate) ** i for i in range(num_years)]

    # Calculate the present value of projected cash flows
    present_values = []
    for i in range(num_years):
        present_value = cash_flows[i] / (1 + discount_rate) ** (i + 1)
        present_values.append(present_value)

    # Calculate the terminal value
    terminal_value = cash_flows[-1] * (1 + terminal_growth_rate) / (discount_rate - terminal_growth_rate)
    terminal_present_value = terminal_value / (1 + discount_rate) ** num_years

    # Sum up the present values and terminal value
    dcf_value = sum(present_values) + terminal_present_value

    return dcf_value

In [5]:
from langchain.agents import load_tools
from langgraph.prebuilt import ToolExecutor
from langchain_community.tools import PolygonLastQuote, PolygonTickerNews, PolygonFinancials, PolygonAggregates
from langchain_community.utilities.polygon import PolygonAPIWrapper

# Create the tools
polygon = PolygonAPIWrapper()
tools = [
    PolygonLastQuote(api_wrapper=polygon),
    PolygonTickerNews(api_wrapper=polygon),
    PolygonFinancials(api_wrapper=polygon),
    PolygonAggregates(api_wrapper=polygon),
] + [discounted_cash_flow]

tool_executor = ToolExecutor(tools)

# Set up the LLM

In [6]:
from langchain.tools.render import format_tool_to_openai_function
from langchain_openai.chat_models import ChatOpenAI

# Choose the LLM that will drive the agent
model = ChatOpenAI(model="gpt-4-0125-preview", streaming=True)

# Bind the functions
functions = [format_tool_to_openai_function(t) for t in tools]
model = model.bind_functions(functions)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.2.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


# Define the agent state

In [7]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

# Define the nodes

In [8]:
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage

# Define the function that determines whether to continue or not
def should_continue(state):
    messages = state['messages']
    last_message = messages[-1]
    # If there is no function call, then we finish
    if "function_call" not in last_message.additional_kwargs:
        return "end"
    # Otherwise if there is, we continue
    else:
        return "continue"

# Define the function that calls the model
def call_model(state):
    messages = state['messages']
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

# Define the function to execute tools
def call_tool(state):
    messages = state['messages']
    # Based on the continue condition
    # we know the last message involves a function call
    last_message = messages[-1]
    # We construct an ToolInvocation from the function_call
    action = ToolInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=json.loads(last_message.additional_kwargs["function_call"]["arguments"]),
    )
    # We call the tool_executor and get back a response
    response = tool_executor.invoke(action)
    # We use the response to create a FunctionMessage
    function_message = FunctionMessage(content=str(response), name=action.tool)
    # We return a list, because this will get added to the existing list
    return {"messages": [function_message]}

# Define the graph

In [9]:
from langgraph.graph import StateGraph, END

# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # END is a special node marking that the graph should finish.
    {
        # If `tools`, then we call the tool node.
        "continue": "action",
        # Otherwise we finish.
        "end": END
    }
)

# We now add a normal edge from `tools` to `agent`.
workflow.add_edge('action', 'agent')

# Finally, we compile it!
app = workflow.compile()

# Run the agent

In [32]:
from langchain_core.messages import HumanMessage

input = {"messages": [HumanMessage(content="What is the intrinsic value of SAMSUNG? Provide me the answer with detailed explanation.")]}
output = app.invoke(input)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
print(output["messages"][-1].content)

Samsung: $688.67 billion.

NVIDIA: $797.86 billion.

Based on the Discounted Cash Flow (DCF) method with the assumptions made:

- **Free Cash Flow (FCF)**: $10,000 (This is an assumed value for illustration purposes).
- **Growth Rate**: 5% (0.05), reflecting an estimated growth rate for the Free Cash Flow.
- **Discount Rate**: 10% (0.1), representing the expected rate of return.
- **Terminal Growth Rate**: 2% (0.02), reflecting long-term economic growth expectations.
- **Projection Period**: 5 years.

The calculated intrinsic value of Samsung, given these assumptions, is approximately **$137,734**.

It's important to note that this value is highly dependent on the assumptions made, especially the initial Free Cash Flow (FCF), which was assumed here for illustrative purposes. The actual intrinsic value of Samsung would require accurate and specific financial data, including the actual Free Cash Flow, tailored growth rates, and other financial metrics specific to Samsung and its industry.

For a more accurate calculation, one would need to delve into Samsung's financial statements to extract the actual Free Cash Flow and consider industry-specific growth rates, discount rates, and economic conditions. Additionally, because Samsung is a large conglomerate with diverse operations, a more segmented analysis might be necessary to accurately assess its intrinsic value.

Based on the Discounted Cash Flow (DCF) model and the assumptions made, the intrinsic value of NVIDIA is approximately $556.74 billion. This value is derived from projecting NVIDIA's future cash flows over the next 5 years with a high growth rate of 15%, discounting those cash flows back to their present value at a discount rate of 10%, and assuming a terminal growth rate of 2.5% thereafter.

It is important to remember that this value is highly sensitive to the assumptions made, particularly the growth rates and the discount rate. Small changes in these inputs can lead to significant variations in the calculated intrinsic value. Additionally, this model does not consider various qualitative factors that may impact NVIDIA's future performance, such as market competition, technological advancements, and regulatory changes.

This valuation should be used as one of many tools in making an investment decision, alongside thorough qualitative analysis and consideration of the broader market and economic conditions.

Based on the Discounted Cash Flow (DCF) method and using the hypothetical assumptions provided:

- **Annual Free Cash Flow (FCF)**: $50 billion
- **Growth Rate**: 5%
- **Discount Rate**: 10%
- **Terminal Growth Rate**: 2%
- **Forecast Period**: 5 years

The intrinsic value of Saudi Aramco (ARAMCO) is approximately $688.67 billion.

**Explanation**:

1. **Free Cash Flow (FCF)**: We started with an assumed annual FCF of $50 billion.

2. **Growth Rate**: We assumed that Aramco's FCF would grow at an annual rate of 5% during the forecast period. This growth rate is applied to project future cash flows.

3. **Discount Rate**: A 10% discount rate was used to account for the risk and the time value of money. This rate is utilized to discount the projected future cash flows back to their present value.

4. **Terminal Growth Rate**: We assumed a 2% perpetual growth rate after the forecast period, reflecting a conservative estimate of Aramco's ability to grow indefinitely beyond the initial 5-year forecast period.

5. **Forecast Period**: The calculation was based on a forecast period of 5 years, during which we applied the specific growth rate to project FCFs.

By summing the present value of the projected cash flows over the forecast period and the present value of the terminal value, we obtained an intrinsic value of approximately $688.67 billion for Aramco.

It's important to note that this calculation is based on hypothetical assumptions. Intrinsic value calculations in real scenarios require accurate and current financial data, and the assumptions about growth rates, discount rates, and terminal growth rates should be carefully considered and justified based on comprehensive analysis.

Based on the Discounted Cash Flow (DCF) method and the assumptions made, the intrinsic value of Alphabet Inc. (GOOGL) is approximately $1,191,151,195,230.86 (or about $1.19 trillion).

This calculation is based on:
- Starting Free Cash Flow (FCF) of approximately $82.3455 billion,
- An FCF growth rate of 5% for the first 5 years,
- A discount rate of 10%,
- A terminal growth rate of 2.5%,
- And a forecast period of 5 years.

It's important to note that the intrinsic value calculation is highly sensitive to the assumptions made regarding the growth rates, discount rate, and the terminal growth rate. Adjusting any of these parameters could significantly alter the outcome. Furthermore, this calculation does not take into account unforeseen circumstances or qualitative factors that may affect Alphabet's future performance.

In [ ]:
print(output["messages"][-1].content)

Based on the Discounted Cash Flow (DCF) method with the assumptions made:

- **Free Cash Flow (FCF)**: $10,000 (This is an assumed value for illustration purposes).
- **Growth Rate**: 5% (0.05), reflecting an estimated growth rate for the Free Cash Flow.
- **Discount Rate**: 10% (0.1), representing the expected rate of return.
- **Terminal Growth Rate**: 2% (0.02), reflecting long-term economic growth expectations.
- **Projection Period**: 5 years.

The calculated intrinsic value of Samsung, given these assumptions, is approximately **$137,734**.

It's important to note that this value is highly dependent on the assumptions made, especially the initial Free Cash Flow (FCF), which was assumed here for illustrative purposes. The actual intrinsic value of Samsung would require accurate and specific financial data, including the actual Free Cash Flow, tailored growth rates, and other financial metrics specific to Samsung and its industry.

For a more accurate calculation, one would need to